In [10]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam


In [11]:
data = pd.read_csv('DeltaBS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price_Change
0,SIRI,2019-12-01,2976.0,1895.0,1081.0,68.1,0.25,-1438.0,-0.30,1458.0,1406.0,1.44
1,NEWT,2015-12-01,50.6,13.1,37.5,4.3,-2.25,34.0,-2.22,7.6,24.1,1.87
2,ADM,2019-12-01,3164.0,2926.0,238.0,-2.2,0.52,-1208.0,-2.08,1512.0,2654.0,5.38
3,PXD,2017-12-01,544.0,-324.0,868.0,0.5,4.94,872.0,4.96,-481.0,-25.0,-7.22
4,TROW,2018-12-01,153.9,106.5,47.4,-8.1,1.96,299.9,1.88,0.0,477.5,-12.61
5,PRU,2013-12-01,22546.0,25777.0,-3231.0,-2.0,-6.62,-3160.0,-6.49,3913.0,10574.0,38.89
6,LEG,2017-12-01,566.7,469.9,96.8,-3.5,0.84,63.8,0.49,291.9,47.7,-1.15
7,ADS,2012-12-01,3019.9,2667.4,352.5,-0.1,7.12,-148.4,-3.15,2387.1,1710.0,40.92
8,PPG,2018-12-01,-523.0,417.0,-940.0,-14.0,-2.49,-982.0,-4.28,870.0,1398.0,-14.59
9,VZ,2015-12-01,11559.0,7393.0,4166.0,-81.6,1.07,-9660.0,-2.82,-3699.0,2384.0,-0.56


In [12]:
prices = data['Price_Change'] 
data = data.drop(['Company', 'Time', 'Price_Change'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price_Change', prices)
data

Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done


,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price_Change
0,0.567761,0.674890,0.524838,0.209157,0.001012,0.478161,0.156527,0.660160,0.609477,1.44
1,0.558627,0.667503,0.518225,0.206439,0.000961,0.488115,0.156493,0.649944,0.600695,1.87
2,0.568348,0.678937,0.519496,0.206162,0.001017,0.479716,0.156496,0.660541,0.617408,5.38
3,0.560168,0.666180,0.523488,0.206277,0.001108,0.493781,0.156620,0.646502,0.600383,-7.22
4,0.558950,0.667870,0.518288,0.205911,0.001047,0.489913,0.156566,0.649890,0.603576,-12.61
5,0.628862,0.768633,0.497513,0.206171,0.000871,0.466516,0.156418,0.677453,0.667741,38.89
6,0.560238,0.669296,0.518601,0.206107,0.001024,0.488316,0.156541,0.651947,0.600845,-1.15
7,0.567898,0.677922,0.520221,0.206252,0.001152,0.486881,0.156477,0.666705,0.611409,40.92
8,0.556836,0.669089,0.512031,0.205659,0.000956,0.481244,0.156457,0.656019,0.609426,-14.59
9,0.594558,0.696471,0.544388,0.202780,0.001028,0.422563,0.156483,0.623835,0.615692,-0.56


In [13]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.56776064 0.67489009 0.52483793 ... 0.15652723 0.66016046 0.60947678]
 [0.55862704 0.66750314 0.51822526 ... 0.1564935  0.649944   0.60069462]
 [0.56834761 0.67893704 0.51949583 ... 0.15649596 0.66054083 0.61740799]
 ...
 [0.56725797 0.67263699 0.52745512 ... 0.1565369  0.65582847 0.60690295]
 [0.5811891  0.69430837 0.52074422 ... 0.15652372 0.67007121 0.60685211]
 [0.56018282 0.67036623 0.51676141 ... 0.15630548 0.65148239 0.60218744]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 4500


[  1.44   1.87   5.38 ...   5.83  77.77 334.81]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 1.44
Total number of y values 4500


In [14]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    

    # Add fully connected layer with a Tanh activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [15]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size = 5)

In [16]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 3600 samples
Epoch 1/1000
3600/3600 [==============================] - 1s 349us/sample - loss: 16.1559 - mae: 16.1559 - mse: 2872.6265
Epoch 2/1000
3600/3600 [==============================] - 1s 309us/sample - loss: 15.9256 - mae: 15.9256 - mse: 2846.4905
Epoch 3/1000
3600/3600 [==============================] - 1s 354us/sample - loss: 15.9308 - mae: 15.9308 - mse: 2841.7151
Epoch 4/1000
3600/3600 [==============================] - 1s 255us/sample - loss: 15.9345 - mae: 15.9345 - mse: 2846.8772
Epoch 5/1000
3600/3600 [==============================] - 1s 251us/sample - loss: 15.9384 - mae: 15.9384 - mse: 2845.6968
Epoch 6/1000
3600/3600 [==============================] - 1s 250us/sample - loss: 15.9290 - mae: 15.9290 - mse: 2845.8523
Epoch 7/1000
3600/3600 [==============================] - 1s 265us/sample - loss: 15.9173 - mae: 15.9173 - mse: 2844.9661
Epoch 8/1000
3600/3600 [==============================] - 1s 389us/sample - loss: 15.9220 - mae: 15.9220 - mse: 2843.2290
Ep

3600/3600 [==============================] - 1s 285us/sample - loss: 15.9141 - mae: 15.9141 - mse: 2845.3667
Epoch 68/1000
3600/3600 [==============================] - 1s 285us/sample - loss: 15.9130 - mae: 15.9130 - mse: 2846.9080
Epoch 69/1000
3600/3600 [==============================] - 1s 269us/sample - loss: 15.9139 - mae: 15.9139 - mse: 2845.5237
Epoch 70/1000
3600/3600 [==============================] - 1s 248us/sample - loss: 15.8955 - mae: 15.8955 - mse: 2844.7190
Epoch 71/1000
3600/3600 [==============================] - 1s 259us/sample - loss: 15.9105 - mae: 15.9105 - mse: 2844.3403
Epoch 72/1000
3600/3600 [==============================] - 1s 256us/sample - loss: 15.9132 - mae: 15.9132 - mse: 2844.0466
Epoch 73/1000
3600/3600 [==============================] - 1s 310us/sample - loss: 15.8889 - mae: 15.8889 - mse: 2842.9861
Epoch 74/1000
3600/3600 [==============================] - 1s 277us/sample - loss: 15.8952 - mae: 15.8952 - mse: 2841.0637
Epoch 75/1000
3600/3600 [=====

3600/3600 [==============================] - 1s 314us/sample - loss: 15.8844 - mae: 15.8844 - mse: 2842.6609
Epoch 134/1000
3600/3600 [==============================] - 1s 352us/sample - loss: 15.8891 - mae: 15.8891 - mse: 2843.6970
Epoch 135/1000
3600/3600 [==============================] - 1s 256us/sample - loss: 15.8929 - mae: 15.8929 - mse: 2845.6113
Epoch 136/1000
3600/3600 [==============================] - 1s 300us/sample - loss: 15.8870 - mae: 15.8870 - mse: 2843.2876
Epoch 137/1000
3600/3600 [==============================] - 1s 286us/sample - loss: 15.8811 - mae: 15.8811 - mse: 2843.2483
Epoch 138/1000
3600/3600 [==============================] - 1s 270us/sample - loss: 15.8971 - mae: 15.8971 - mse: 2846.7996
Epoch 139/1000
3600/3600 [==============================] - 1s 271us/sample - loss: 15.8937 - mae: 15.8937 - mse: 2840.8342
Epoch 140/1000
3600/3600 [==============================] - 1s 323us/sample - loss: 15.8844 - mae: 15.8844 - mse: 2844.4973
Epoch 141/1000
3600/360

3600/3600 [==============================] - 1s 331us/sample - loss: 15.8764 - mae: 15.8764 - mse: 2842.5117
Epoch 200/1000
3600/3600 [==============================] - 1s 349us/sample - loss: 15.8798 - mae: 15.8798 - mse: 2842.0964
Epoch 201/1000
3600/3600 [==============================] - 1s 326us/sample - loss: 15.8727 - mae: 15.8727 - mse: 2841.2415
Epoch 202/1000
3600/3600 [==============================] - 1s 326us/sample - loss: 15.8844 - mae: 15.8844 - mse: 2841.7351
Epoch 203/1000
3600/3600 [==============================] - 1s 327us/sample - loss: 15.8907 - mae: 15.8907 - mse: 2844.0542
Epoch 204/1000
3600/3600 [==============================] - 1s 364us/sample - loss: 15.8799 - mae: 15.8799 - mse: 2842.1650
Epoch 205/1000
3600/3600 [==============================] - 1s 338us/sample - loss: 15.8802 - mae: 15.8802 - mse: 2844.7200
Epoch 206/1000
3600/3600 [==============================] - 1s 375us/sample - loss: 15.8818 - mae: 15.8818 - mse: 2842.9089
Epoch 207/1000
3600/360

3600/3600 [==============================] - 1s 335us/sample - loss: 15.8519 - mae: 15.8519 - mse: 2838.0396
Epoch 266/1000
3600/3600 [==============================] - 1s 398us/sample - loss: 15.8808 - mae: 15.8808 - mse: 2843.0347
Epoch 267/1000
3600/3600 [==============================] - 1s 375us/sample - loss: 15.8713 - mae: 15.8713 - mse: 2843.4165
Epoch 268/1000
3600/3600 [==============================] - 1s 331us/sample - loss: 15.8672 - mae: 15.8672 - mse: 2840.1204
Epoch 269/1000
3600/3600 [==============================] - 1s 334us/sample - loss: 15.8697 - mae: 15.8697 - mse: 2839.9648
Epoch 270/1000
3600/3600 [==============================] - 1s 354us/sample - loss: 15.8644 - mae: 15.8644 - mse: 2842.6345
Epoch 271/1000
3600/3600 [==============================] - 2s 481us/sample - loss: 15.8741 - mae: 15.8741 - mse: 2841.7197
Epoch 272/1000
3600/3600 [==============================] - 1s 378us/sample - loss: 15.8666 - mae: 15.8666 - mse: 2839.4338
Epoch 273/1000
3600/360

...

3600/3600 [==============================] - 1s 371us/sample - loss: 15.5113 - mae: 15.5113 - mse: 2615.4963
Epoch 726/1000
3600/3600 [==============================] - 1s 384us/sample - loss: 15.4998 - mae: 15.4998 - mse: 2611.3301
Epoch 727/1000
3600/3600 [==============================] - 1s 412us/sample - loss: 15.4976 - mae: 15.4976 - mse: 2611.0608
Epoch 728/1000
3600/3600 [==============================] - 2s 420us/sample - loss: 15.4854 - mae: 15.4854 - mse: 2613.3210
Epoch 729/1000
3600/3600 [==============================] - 2s 461us/sample - loss: 15.4945 - mae: 15.4945 - mse: 2613.3484
Epoch 730/1000
3600/3600 [==============================] - 2s 504us/sample - loss: 15.4903 - mae: 15.4903 - mse: 2613.0510
Epoch 731/1000
3600/3600 [==============================] - 1s 409us/sample - loss: 15.4997 - mae: 15.4997 - mse: 2614.2114
Epoch 732/1000
3600/3600 [==============================] - 1s 363us/sample - loss: 15.4870 - mae: 15.4870 - mse: 2614.3491
Epoch 733/1000
3600/360

3600/3600 [==============================] - 1s 382us/sample - loss: 15.4982 - mae: 15.4982 - mse: 2613.7153
Epoch 792/1000
3600/3600 [==============================] - 2s 438us/sample - loss: 15.4977 - mae: 15.4977 - mse: 2612.8738
Epoch 793/1000
3600/3600 [==============================] - 1s 390us/sample - loss: 15.4814 - mae: 15.4814 - mse: 2612.6309
Epoch 794/1000
3600/3600 [==============================] - 1s 373us/sample - loss: 15.4885 - mae: 15.4885 - mse: 2608.8772
Epoch 795/1000
3600/3600 [==============================] - 1s 374us/sample - loss: 15.4898 - mae: 15.4898 - mse: 2612.6619
Epoch 796/1000
3600/3600 [==============================] - 2s 428us/sample - loss: 15.4872 - mae: 15.4872 - mse: 2612.5444
Epoch 797/1000
3600/3600 [==============================] - 1s 376us/sample - loss: 15.4725 - mae: 15.4725 - mse: 2611.3374
Epoch 798/1000
3600/3600 [==============================] - 1s 397us/sample - loss: 15.4641 - mae: 15.4641 - mse: 2610.3562
Epoch 799/1000
3600/360

3600/3600 [==============================] - 2s 551us/sample - loss: 15.4882 - mae: 15.4882 - mse: 2609.6592
Epoch 857/1000
3600/3600 [==============================] - 2s 604us/sample - loss: 15.4669 - mae: 15.4669 - mse: 2611.0012
Epoch 858/1000
3600/3600 [==============================] - 2s 544us/sample - loss: 15.4819 - mae: 15.4819 - mse: 2613.3577
Epoch 859/1000
3600/3600 [==============================] - 1s 404us/sample - loss: 15.4741 - mae: 15.4741 - mse: 2610.9280
Epoch 860/1000
3600/3600 [==============================] - 1s 382us/sample - loss: 15.4884 - mae: 15.4884 - mse: 2610.6453
Epoch 861/1000
3600/3600 [==============================] - 1s 385us/sample - loss: 15.4651 - mae: 15.4651 - mse: 2610.4739
Epoch 862/1000
3600/3600 [==============================] - 1s 362us/sample - loss: 15.4941 - mae: 15.4941 - mse: 2612.2971
Epoch 863/1000
3600/3600 [==============================] - 1s 391us/sample - loss: 15.4884 - mae: 15.4884 - mse: 2613.3652
Epoch 864/1000
3600/360

3600/3600 [==============================] - 1s 371us/sample - loss: 15.4720 - mae: 15.4720 - mse: 2609.7427
Epoch 922/1000
3600/3600 [==============================] - 1s 395us/sample - loss: 15.4950 - mae: 15.4951 - mse: 2608.5405
Epoch 923/1000
3600/3600 [==============================] - 1s 380us/sample - loss: 15.4661 - mae: 15.4661 - mse: 2610.6487
Epoch 924/1000
3600/3600 [==============================] - 1s 359us/sample - loss: 15.4823 - mae: 15.4823 - mse: 2609.2380
Epoch 925/1000
3600/3600 [==============================] - 1s 367us/sample - loss: 15.4777 - mae: 15.4778 - mse: 2614.2219
Epoch 926/1000
3600/3600 [==============================] - 1s 394us/sample - loss: 15.4928 - mae: 15.4928 - mse: 2611.6677
Epoch 927/1000
3600/3600 [==============================] - 1s 377us/sample - loss: 15.4845 - mae: 15.4845 - mse: 2612.3450
Epoch 928/1000
3600/3600 [==============================] - 1s 349us/sample - loss: 15.4739 - mae: 15.4739 - mse: 2611.5322
Epoch 929/1000
3600/360

3600/3600 [==============================] - 2s 454us/sample - loss: 15.4650 - mae: 15.4650 - mse: 2610.3308
Epoch 988/1000
3600/3600 [==============================] - 1s 408us/sample - loss: 15.4821 - mae: 15.4821 - mse: 2611.4058
Epoch 989/1000
3600/3600 [==============================] - 1s 360us/sample - loss: 15.4703 - mae: 15.4703 - mse: 2609.1941
Epoch 990/1000
3600/3600 [==============================] - 2s 418us/sample - loss: 15.4805 - mae: 15.4805 - mse: 2613.3904
Epoch 991/1000
3600/3600 [==============================] - 2s 437us/sample - loss: 15.4719 - mae: 15.4719 - mse: 2611.2051
Epoch 992/1000
3600/3600 [==============================] - 1s 369us/sample - loss: 15.4680 - mae: 15.4680 - mse: 2611.9495
Epoch 993/1000
3600/3600 [==============================] - 2s 429us/sample - loss: 15.4527 - mae: 15.4527 - mse: 2609.2568
Epoch 994/1000
3600/3600 [==============================] - 2s 426us/sample - loss: 15.4583 - mae: 15.4583 - mse: 2610.2932
Epoch 995/1000
3600/360

In [17]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-13.85323507 -16.21910616 -14.79944383 -17.10114784 -15.01574857]
Mean MAE: -15.397736293872196
Standard Deviation of the mean: 1.137155388798084
